In [10]:
# Import numpy and TF backend
import numpy as np
import tensorflow as tf
tf.python.control_flow_ops = tf

# Import Keras 
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Cropping2D, Lambda
from keras import backend as K

# Import cv
import cv2
import csv



In [11]:
# Functions

# get ImageMat for the given file location in the Data log
def process_image(imageField):
    imagePath = imageField
    fileName = imagePath.split('/')[-1]
    fileNamewithFolder = './data/IMG/' + fileName
    imageMat = cv2.imread(fileNamewithFolder)
    
    return imageMat

In [12]:
lines = []

with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
    print(len(lines))
    
# Traverse thru the lines and extract images (X values) and steering value (y value)
images = []
msmts = []
for line in lines:
    # First element is the image
    imageMat = process_image(line[0])
    images.append(imageMat)
    # 4th element is the steering info
    steering_center = float(line[3])
    msmts.append(steering_center)
    
    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    imageMat =  process_image(line[1])
    images.append(imageMat)
    msmts.append(steering_left)
    
    imageMat =  process_image(line[2])
    images.append(imageMat)
    msmts.append(steering_right)

# Find the existing length of images
list_len = len(msmts)
for i in range(0, list_len):
    
    # Now add the image flip side to reduce the left bias
    image_flipped = np.fliplr(images[i])
    msmt_flipped = -1 * msmts[i]
    images.append(image_flipped)
    msmts.append(msmt_flipped)

print(len(msmts))

4706
28236


In [13]:
# Set the traing data
X_train = np.array(images)
Y_train = np.array(msmts)


In [14]:
# TODO: Build the Fully Connected Neural Network in Keras Here
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Convolution2D(24, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(32, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(48, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())

#2nd Layer - Add a fully connected layer
model.add(Dense(128))
#3rd Layer - Add a ReLU activation layer
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(42))
model.add(Dense(10))
#4th Layer - Add a fully connected layer - one node at the end to predict steering (basically regression)
model.add(Dense(1))

In [15]:
# Train the model
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, Y_train, nb_epoch=5, validation_split=0.2, shuffle=True)

model.save('bc_model_jul13.h5')

Train on 22588 samples, validate on 5648 samples
Epoch 1/5
22588/22588 [==============================] - 39s - loss: 0.0592 - val_loss: 0.0676
Epoch 2/5
22588/22588 [==============================] - 37s - loss: 0.0537 - val_loss: 0.0603
Epoch 3/5
22588/22588 [==============================] - 37s - loss: 0.0492 - val_loss: 0.0576
Epoch 4/5
22588/22588 [==============================] - 36s - loss: 0.0463 - val_loss: 0.0637
Epoch 5/5
22588/22588 [==============================] - 36s - loss: 0.0433 - val_loss: 0.0667
